In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['font.family'] = 'Malgun Gothic' # 한글 폰트 설정
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import torch #cuda용 
import warnings # 경고 메시지 무시
warnings.filterwarnings('ignore')
from datetime import datetime
import math
from tqdm import tqdm

In [2]:
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"PyTorch CUDA 버전: {torch.version.cuda}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
else:
    print("CPU 버전이 설치됨")

CUDA 사용 가능: True
PyTorch CUDA 버전: 11.8
GPU 이름: NVIDIA GeForce GTX 1650


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

# 데이터 로드 및 분할
building = pd.read_csv('../data/building_info.csv')
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

train['is_train'] = 1
test['is_train'] = 0

# 전체 데이터 합치기
all_data = pd.concat([train, test], ignore_index=True)

# train과 test 분리
train_df = all_data[all_data['is_train'] == 1].drop('is_train', axis=1)
test_df = all_data[all_data['is_train'] == 0].drop('is_train', axis=1)

# 시간 특성 생성
date = pd.to_datetime(train_df['일시'])
train_df['hour'] = date.dt.hour
train_df['day'] = date.dt.weekday
train_df['month'] = date.dt.month
train_df['week'] = date.dt.isocalendar().week

date = pd.to_datetime(test_df['일시'])
test_df['hour'] = date.dt.hour
test_df['day'] = date.dt.weekday
test_df['month'] = date.dt.month
test_df['week'] = date.dt.isocalendar().week

# 건물정보 전처리
cols = ['태양광용량(kW)','ESS저장용량(kWh)','PCS용량(kW)']
for col in cols:
    building[col] = building[col].apply(lambda x: 0 if x == '-' else float(x))

# train 데이터 결측치 처리
train_ft = train_df.drop(columns=['num_date_time', '일시','일조(hr)', '일사(MJ/m2)'])
test_ft = test_df.drop(columns=['num_date_time', '일시'])

train_ft['강수량(mm)'] = train_ft['강수량(mm)'].fillna(0)

# 풍속 결측치 처리
for i in train_ft[train_ft['풍속(m/s)'].isnull()].index:
    month = train_ft.iloc[i]['month']
    hour = train_ft.iloc[i]['hour']
    num = train_ft.iloc[i]['건물번호']
    data_mean = train_ft[(train_ft['건물번호']==num)&(train_ft['month']==month) & (train_ft['hour']==hour)]['풍속(m/s)'].describe()['mean']
    train_ft.loc[i,'풍속(m/s)'] = data_mean

# 습도 결측치 처리
for i in train_ft[train_ft['습도(%)'].isnull()].index:
    month = train_ft.iloc[i]['month']
    hour = train_ft.iloc[i]['hour']
    num = train_ft.iloc[i]['건물번호']
    data_mean = train_ft[(train_ft['건물번호']==num)&(train_ft['month']==month) & (train_ft['hour']==hour)]['습도(%)'].describe()['mean']
    train_ft.loc[i,'습도(%)'] = data_mean

# 건물정보 합치기
train_ft = train_ft.merge(building, how='left', on='건물번호')
test_ft = test_ft.merge(building, how='left', on='건물번호')

# 범주형 인코딩
conv_dict = dict(zip(list(train_ft['건물유형'].unique()), range(len(train_ft['건물유형'].unique()))))
train_ft['건물유형'] = train_ft['건물유형'].map(conv_dict)
test_ft['건물유형'] = test_ft['건물유형'].map(conv_dict)

# 타겟 생성
target = train_df['전력소비량(kWh)']

# 공휴일 특성
train_ft['holiday'] = train_ft.apply(lambda x: 0 if x['day'] < 5 else 1, axis=1)
test_ft['holiday'] = test_ft.apply(lambda x: 0 if x['day'] < 5 else 1, axis=1)

# 시간 순환 특성
train_ft['sin_time'] = np.sin(2*np.pi*train_ft.hour/24)
train_ft['cos_time'] = np.cos(2*np.pi*train_ft.hour/24)
test_ft['sin_time'] = np.sin(2*np.pi*test_ft.hour/24)
test_ft['cos_time'] = np.cos(2*np.pi*test_ft.hour/24)

# 컬럼명 확인 및 디버깅
print("train_ft 컬럼들:")
print(train_ft.columns.tolist())
print("\ntest_ft 컬럼들:")
print(test_ft.columns.tolist())

# 가능한 기온 컬럼명들 확인
temp_cols = [col for col in train_ft.columns if '기온' in col or 'temp' in col.lower()]
humidity_cols = [col for col in train_ft.columns if '습도' in col or 'humid' in col.lower()]

print(f"\n기온 관련 컬럼: {temp_cols}")
print(f"습도 관련 컬럼: {humidity_cols}")

# 실제 컬럼명 사용
if temp_cols and humidity_cols:
    temp_col = temp_cols[0]  # 첫 번째 기온 컬럼
    humidity_col = humidity_cols[0]  # 첫 번째 습도 컬럼
    
    print(f"\n사용할 컬럼: 기온={temp_col}, 습도={humidity_col}")
    
    # 불쾌지수
    train_ft['THI'] = 9/5*train_ft[temp_col] - 0.55*(1-train_ft[humidity_col]/100)*(9/5*train_ft[temp_col]-26)+32
    test_ft['THI'] = 9/5*test_ft[temp_col] - 0.55*(1-test_ft[humidity_col]/100)*(9/5*test_ft[temp_col]-26)+32
    
    train_ft['THI'] = pd.cut(train_ft['THI'], bins=[0, 68, 75, 80, 200], labels=[1,2,3,4])
    train_ft['THI'] = train_ft['THI'].astype(int)
    test_ft['THI'] = pd.cut(test_ft['THI'], bins=[0, 68, 75, 80, 200], labels=[1,2,3,4])
    test_ft['THI'] = test_ft['THI'].astype(int)
    
    # CDH 함수
    def CDH(xs):
        ys = []
        for i in range(len(xs)):
            if i < 11:
                ys.append(np.sum(xs[:(i+1)]-26))
            else:
                ys.append(np.sum(xs[(i-11):(i+1)]-26))
        return np.array(ys)
    
    # CDH 특성 생성
    cdhs = np.array([])
    for num in range(1, 101, 1):
        temp = train_ft[train_ft['건물번호'] == num]
        cdh = CDH(temp[temp_col].values)
        cdhs = np.concatenate([cdhs, cdh])
    train_ft['CDH'] = cdhs
    
    cdhs = np.array([])
    for num in range(1, 101, 1):
        temp = test_ft[test_ft['건물번호'] == num]
        cdh = CDH(temp[temp_col].values)
        cdhs = np.concatenate([cdhs, cdh])
    test_ft['CDH'] = cdhs
else:
    print("기온 또는 습도 컬럼을 찾을 수 없습니다. 컬럼명을 직접 확인해주세요.")

# 요일 시간별 사용량 특성들
power_mean = pd.pivot_table(train_ft, values='전력소비량(kWh)', index=['건물번호', 'hour', 'day','month'], aggfunc=np.mean).reset_index()
power_mean = power_mean.drop_duplicates(subset=['건물번호', 'hour', 'day'], keep='first')

train_ft = train_ft.merge(power_mean[['건물번호', 'hour', 'day', '전력소비량(kWh)']], on=['건물번호', 'hour', 'day'], how='left', suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'day_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', 'day', '전력소비량(kWh)']], on=['건물번호', 'hour', 'day'], how='left', suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_mean'}, inplace=True)

# 월-시간별 평균
power_mean = pd.pivot_table(train_ft, values='전력소비량(kWh)', index=['건물번호', 'hour', 'day','month'], aggfunc=np.mean).reset_index()
power_mean = power_mean.drop_duplicates(subset=['건물번호', 'hour', 'month'], keep='first')

train_ft = train_ft.merge(power_mean[['건물번호', 'hour', 'month', '전력소비량(kWh)']], on=['건물번호', 'hour', 'month'], how='left', suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'month_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', 'month', '전력소비량(kWh)']], on=['건물번호', 'hour', 'month'], how='left', suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'month_hour_mean'}, inplace=True)

# 시간별 평균/표준편차/중앙값
power_mean = pd.pivot_table(train_ft, values='전력소비량(kWh)', index=['건물번호', 'hour'], aggfunc=np.mean).reset_index()
train_ft = train_ft.merge(power_mean[['건물번호', 'hour', '전력소비량(kWh)']], on=['건물번호', 'hour'], how='left', suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['건물번호', 'hour', '전력소비량(kWh)']], on=['건물번호', 'hour'], how='left', suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'hour_mean'}, inplace=True)

power_std = pd.pivot_table(train_ft, values='전력소비량(kWh)', index=['건물번호', 'hour'], aggfunc=np.std).reset_index()
train_ft = train_ft.merge(power_std[['건물번호', 'hour', '전력소비량(kWh)']], on=['건물번호', 'hour'], how='left', suffixes=('', '_std'))
train_ft.rename(columns={'전력소비량(kWh)_std': 'hour_std'}, inplace=True)

test_ft = test_ft.merge(power_std[['건물번호', 'hour', '전력소비량(kWh)']], on=['건물번호', 'hour'], how='left', suffixes=('', '_std'))
test_ft.rename(columns={'전력소비량(kWh)': 'hour_std'}, inplace=True)

power_std = pd.pivot_table(train_ft, values='전력소비량(kWh)', index=['건물번호', 'hour', 'day'], aggfunc=np.std).reset_index()
train_ft = train_ft.merge(power_std[['건물번호', 'hour', 'day', '전력소비량(kWh)']], on=['건물번호', 'hour', 'day'], how='left', suffixes=('', '_std'))
train_ft.rename(columns={'전력소비량(kWh)_std': 'day_hour_std'}, inplace=True)

test_ft = test_ft.merge(power_std[['건물번호', 'hour', 'day', '전력소비량(kWh)']], on=['건물번호', 'hour', 'day'], how='left', suffixes=('', '_std'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_std'}, inplace=True)

power_median = pd.pivot_table(train_ft, values='전력소비량(kWh)', index=['건물번호', 'hour', 'day'], aggfunc=np.median).reset_index()
train_ft = train_ft.merge(power_median[['건물번호', 'hour', 'day', '전력소비량(kWh)']], on=['건물번호', 'hour', 'day'], how='left', suffixes=('', '_median'))
train_ft.rename(columns={'전력소비량(kWh)_median': 'day_hour_median'}, inplace=True)

test_ft = test_ft.merge(power_median[['건물번호', 'hour', 'day', '전력소비량(kWh)']], on=['건물번호', 'hour', 'day'], how='left', suffixes=('', '_median'))
test_ft.rename(columns={'전력소비량(kWh)': 'day_hour_median'}, inplace=True)

# 불필요 컬럼 제거
train_ft = train_ft.drop(columns=['holiday', '건물유형', '강수량(mm)', '풍속(m/s)','전력소비량(kWh)'])
test_ft = test_ft.drop(columns=['holiday', '건물유형', '강수량(mm)', '풍속(m/s)'])

# 실제 존재하는 컬럼만 선택
print("최종 train_ft 컬럼들:")
print(train_ft.columns.tolist())

# 기본 컬럼들
base_cols = ['건물번호', 'hour', 'day', 'month', 'week', '연면적(m2)',
           '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'sin_time',
           'cos_time', 'day_hour_mean', 'month_hour_mean','hour_mean', 'hour_std','day_hour_std',
           'day_hour_median']

# 실제 존재하는 컬럼만 필터링
existing_cols = [col for col in base_cols if col in train_ft.columns]

# 기온, 습도, THI, CDH 컬럼 추가 (존재하는 경우만)
for col in train_ft.columns:
    if any(keyword in col for keyword in ['기온', '습도', 'THI', 'CDH', '이동평균']):
        if col not in existing_cols:
            existing_cols.append(col)

print(f"선택된 컬럼들: {existing_cols}")

train_ft = train_ft[existing_cols]
test_ft = test_ft[existing_cols]

train_ft['전력소비량(kWh)'] = train_df['전력소비량(kWh)']

# 클러스터링
train_ft['holiday'] = train_ft.apply(lambda x: 0 if x['day'] < 5 else 1, axis=1)
test_ft['holiday'] = test_ft.apply(lambda x: 0 if x['day'] < 5 else 1, axis=1)

weekday_mean = train_ft[train_ft.holiday==0].pivot_table(values='전력소비량(kWh)', index='건물번호', columns='hour', aggfunc='first')
weekend_mean = train_ft[train_ft.holiday==1].pivot_table(values='전력소비량(kWh)', index='건물번호', columns='hour', aggfunc='first')

tmp = pd.merge(weekday_mean, weekend_mean, how='left', on='건물번호')

SEED = 42
kmeans = KMeans(n_clusters=5, random_state=SEED, n_init='auto')
kmeans.fit(tmp)

cluster_ = kmeans.predict(tmp)
tmp['cluster'] = cluster_

train_ft = train_ft.merge(tmp[['cluster']], how='left', on='건물번호')
test_ft = test_ft.merge(tmp[['cluster']], how='left', on='건물번호')

train_ft = train_ft.drop(columns=['holiday'])
test_ft = test_ft.drop(columns=['holiday'])

power_mean = pd.pivot_table(train_ft, values='전력소비량(kWh)', index=['cluster', 'hour', 'day','month'], aggfunc=np.mean).reset_index()
power_mean = power_mean.drop_duplicates(subset=['cluster', 'hour', 'day'], keep='first')

train_ft = train_ft.merge(power_mean[['cluster', 'hour', 'day', '전력소비량(kWh)']], on=['cluster', 'hour', 'day'], how='left', suffixes=('', '_mean'))
train_ft.rename(columns={'전력소비량(kWh)_mean': 'cluster_day_hour_mean'}, inplace=True)

test_ft = test_ft.merge(power_mean[['cluster', 'hour', 'day', '전력소비량(kWh)']], on=['cluster', 'hour', 'day'], how='left', suffixes=('', '_mean'))
test_ft.rename(columns={'전력소비량(kWh)': 'cluster_day_hour_mean'}, inplace=True)

train_ft = train_ft.drop(columns=['전력소비량(kWh)'])

# 이동평균 (기온, 습도 컬럼이 있는 경우만)
if temp_cols and humidity_cols:
    temp_col = temp_cols[0]
    humidity_col = humidity_cols[0]
    
    # 이동평균
    window_size = 96
    train_ft['기온_4일_이동평균'] = train_ft.groupby('건물번호')[temp_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    train_ft['습도_4일_이동평균'] = train_ft.groupby('건물번호')[humidity_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    
    window_size = 168
    train_ft['기온_7일_이동평균'] = train_ft.groupby('건물번호')[temp_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    train_ft['습도_7일_이동평균'] = train_ft.groupby('건물번호')[humidity_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    
    # 이동평균을 위해 train 데이터를 test에 추가
    for i in range(1, 101):
        test_ft = pd.concat([train_ft[train_ft['건물번호']==i], test_ft], axis=0)
    
    window_size = 96
    test_ft['기온_4일_이동평균'] = test_ft.groupby('건물번호')[temp_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    test_ft['습도_4일_이동평균'] = test_ft.groupby('건물번호')[humidity_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    
    window_size = 168
    test_ft['기온_7일_이동평균'] = test_ft.groupby('건물번호')[temp_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    test_ft['습도_7일_이동평균'] = test_ft.groupby('건물번호')[humidity_col].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())
    
    # 실제 test 데이터만 남기기 (train 길이만큼 제거)
    test_ft = test_ft.tail(len(test_df))

print("전처리 완료!")
print(f"Train shape: {train_ft.shape}")
print(f"Test shape: {test_ft.shape}")
print(f"Target shape: {target.shape}")

# 머신러닝 학습용 데이터 준비
X_train = train_ft.copy()
y_train = target.copy()
X_test = test_ft.copy()

print("\n머신러닝 학습 준비 완료!")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")  
print(f"X_test shape: {X_test.shape}")

# 결측치 확인
print(f"\nX_train 결측치: {X_train.isnull().sum().sum()}")
print(f"X_test 결측치: {X_test.isnull().sum().sum()}")
print(f"y_train 결측치: {y_train.isnull().sum()}")

train_ft 컬럼들:
['건물번호', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '전력소비량(kWh)', 'hour', 'day', 'month', 'week', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'holiday', 'sin_time', 'cos_time']

test_ft 컬럼들:
['건물번호', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', 'hour', 'day', 'month', 'week', '건물유형', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'holiday', 'sin_time', 'cos_time']

기온 관련 컬럼: ['기온(°C)']
습도 관련 컬럼: ['습도(%)']

사용할 컬럼: 기온=기온(°C), 습도=습도(%)
최종 train_ft 컬럼들:
['건물번호', '기온(°C)', '습도(%)', 'hour', 'day', 'month', 'week', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'sin_time', 'cos_time', 'THI', 'CDH', 'day_hour_mean', 'month_hour_mean', 'hour_mean', 'hour_std', 'day_hour_std', 'day_hour_median']
선택된 컬럼들: ['건물번호', 'hour', 'day', 'month', 'week', '연면적(m2)', '냉방면적(m2)', '태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)', 'sin_time', 'cos_time', 'day_hour_mean', 'month_hour_mean', 'hour_mean', 'hour_